In [1]:
import numpy as np
import pandas as pd

In [131]:
operational_hours = 8760

# Commodity Trends

In [137]:
fname = 'Excel_input.xlsx'

In [192]:
# create bulk commodities class
class bulk_commodities:
    def __init__(self, fname, commodity_name):
        self.commodity_name = commodity_name
        pass
    
    def get_forecast(self):
        total_forecast  = pd.read_excel(fname, 'TF')
        years           = total_forecast["Year"]
        self.start_year = years[0]
        self.stop_year  = years[len(years)-1]
        self.n_years    = len(years)
        self.forecast   = total_forecast[self.commodity_name]
        
    def modal_split(self,):
        modal_split               = pd.read_excel(fname, 'Vessel distribution')
        if self.commodity_name == "Maize":
            self.handysize_throughput = modal_split["Handysize maize"]
            self.handymax_throughput  = modal_split["Handymax maize"]
            self.panamax_throughput   = modal_split["Panamax maize"]
        if self.commodity_name == "Soybeans":
            self.handysize_throughput = modal_split["Handysize soybeans"]
            self.handymax_throughput  = modal_split["Handymax soybeans"]
            self.panamax_throughput   = modal_split["Panamax soybeans"]
        if self.commodity_name == "Wheat":
            self.handysize_throughput = modal_split["Handysize wheat"]
            self.handymax_throughput  = modal_split["Handymax wheat"]
            self.panamax_throughput   = modal_split["Panamax wheat"]

In [143]:
# create bulk commodities class
class bulk_modal_splits:
    def __init__(self, fname):
        self.split_forecasts = pd.read_excel(fname, 'Vessel distribution')
        self.years = self.split_forecasts["Year"]
        #self.start_year = years[0]
        #self.stop_year  = years[len(years)-1]
        #self.n_years    = len(years)
        
    def get_forecast(self,commodity_name):
        # the hard coded columns 1:4, 4:7 and 7:10 indicate that the datamodel is not yet ideal
        if commodity_name == "Maize":
            return self.split_forecasts[modal_splits.split_forecasts.columns[1:4]]
        if commodity_name == "Soybeans":
            return self.split_forecasts[modal_splits.split_forecasts.columns[4:7]]
        if commodity_name == "Wheat":
            return self.split_forecasts[modal_splits.split_forecasts.columns[7:10]]

In [193]:
# get commodity forecast from XLS
maize = bulk_commodities(fname, "Maize")

# extract specifics for Maize
#tf_maize = commodities.get_forecast("Maize")

In [194]:
maize.__dict__

{'commodity_name': 'Maize'}

In [195]:
maize.get_forecast()
maize.modal_split()

KeyError: 'Handymax maize'

In [ ]:
# get modal split forecast from XLS
modal_splits = bulk_modal_splits(fname)

# extract specifics for Maize
tf_mod_split = modal_splits.get_forecast("Maize")

# Vessels

In [3]:
# create vessel class
class vessel_properties_mixin(object):
    def __init__(self, 
                 vessel_type, call_size, LOA, draft, beam, max_cranes, all_turn_time, mooring_time, demurrage, 
                 *args, **kwargs):
        super().__init__(*args, **kwargs)
        "initialize"
        self.vessel_type   = vessel_type
        self.call_size     = call_size 
        self.LOA           = LOA
        self.draft         = draft
        self.beam          = beam
        self.max_cranes    = max_cranes
        self.all_turn_time = all_turn_time
        self.mooring_time  = mooring_time
        self.demurrage     = demurrage
    
    
# Initial data set, data from Excel_input.xlsx
handysize_data = {"vessel_type": 'Handysize', "call_size": 35000, 
                  "LOA": 130, "draft": 10, "beam": 24, "max_cranes": 2, 
                  "all_turn_time": 24, "mooring_time": 3, "demurrage": 600}

handymax_data = {"vessel_type": 'Handymax', "call_size": 50000, 
                  "LOA": 180, "draft": 11.5, "beam": 28, "max_cranes": 2, 
                  "all_turn_time": 24, "mooring_time": 3, "demurrage": 750}

panamax_data = {"vessel_type": 'Panamax', "call_size": 65000, 
                  "LOA": 220, "draft": 13, "beam": 32.2, "max_cranes": 3, 
                  "all_turn_time": 36, "mooring_time": 4, "demurrage": 730} 

In [129]:
# create vessel class (will be moved to Port Model revamp notebook)
class vessel(vessel_properties_mixin):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
# create objects
handysize = vessel(**handysize_data)
handymax = vessel(**handymax_data)
panamax = vessel(**panamax_data)

# Terminal Infrastructure

## Quay wall

In [20]:
# create quay wall class
class quay_wall_properties_mixin(object):
    def __init__(self, ownership, lifespan, unit_rate, mobilisation_min, mobilisation_perc, maintenance_perc, insurance_perc, length, delta, depth, max_cranes_per_berth, *args, **kwargs):
        super().__init__(*args, **kwargs)
        "initialize"
        self.ownership            = ownership
        self.lifespan             = lifespan
        self.unit_rate            = unit_rate
        self.mobilisation_min     = mobilisation_min
        self.mobilisation_perc    = mobilisation_perc
        self.maintenance_perc     = maintenance_perc
        self.insurance_perc       = insurance_perc

# Initial data set, data from Excel_input.xlsx
quay_data = {"ownership": 'Port authority', "lifespan": 50, "unit_rate": 100000, "mobilisation_min": 2500000,
             "mobilisation_perc": 0.02, "maintenance_perc": 0.01, "insurance": 0.01,} 

In [78]:
# create quay class (will be moved to Port Model revamp notebook)
class quay_wall(quay_wall_properties_mixin):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    
    def original_value_calc(self):
        self.original_value = int(self.length * self.unit_rate) 
        
    def mobilisation_calc():
        self.mobilisation = int(max((self.delta * self.unit_rate * self.mobilisation_perc), self.mobilisation_min))
        
    def maintenance_calc(self):
        self.maintenance = int(self.original_value * self.maintenance_perc)
        
    def insurance_calc(self):
        self.insurance = int(self.original_value * self.insurance_perc)
        
    def delta_calc(self):
        self.delta = 0
        
    def calcs(self):
        self.original_value_calc()
        self.mobilisation_calc()
        self.maintenance_calc()
        self.insurance_calc()
        self.delta_calc()

In [ ]:
# create objects
quay = quay_wall(**quay_data)

## Cranes (cyclic)

In [74]:
# create cyclic loader class
class cyclic_properties_mixin(object):
    def __init__(self, ownership, lifespan, unit_rate, mobilisation_perc, maintenance_perc, insurance_perc, consumption, crew, unloader_type, lifting_capacity, hourly_cycles, eff_fact, *args, **kwargs):
        super().__init__(*args, **kwargs)
        "initialize"
        self.ownership            = ownership
        self.lifespan             = lifespan
        self.unit_rate            = unit_rate
        self.mobilisation_perc    = mobilisation_perc
        self.mobilisation         = mobilisation_perc * unit_rate
        self.maintenance_perc     = maintenance_perc
        self.insurance_perc       = insurance_perc
        self.consumption          = consumption
        self.crew                 = crew
        self.unloader_type        = unloader_type
        self.lifting_capacity     = lifting_capacity
        self.hourly_cycles        = hourly_cycles
        self.eff_fact             = eff_fact 
        self.payload              = int(0.70 * self.lifting_capacity)      #Source: Nemag
        self.peak_capacity        = int(self.payload * self.hourly_cycles) #Or as direct input 
        self.effective_capacity   = int(eff_fact * self.peak_capacity)     #Source: TATA steel

# Initial data set, data from Excel_input.xlsx
gantry_data        = {"ownership": 'Terminal operator', "lifespan": 40, "unit_rate": 19500000,"mobilisation_perc": 0.15, 
                      "maintenance_perc": 0.02, "insurance_perc": 0.01,"consumption": 0, "crew": 3, 
                      "unloader_type": 'Gantry crane', "lifting_capacity": 70, "hourly_cycles": 60, "eff_fact": 0.55}

harbour_crane_data = {"ownership": 'Terminal operator', "lifespan": 40, "unit_rate": 1400000, "mobilisation_perc": 0.15, 
                      "maintenance_perc": 0.02, "insurance_perc": 0.01,"consumption": 0, "crew": 3, 
                      "unloader_type": 'Harbour crane crane', "lifting_capacity": 40, "hourly_cycles": 40, "eff_fact": 0.55}

mobile_crane_data  = {"ownership": 'Terminal operator', "lifespan": 20, "unit_rate": 3325000,"mobilisation_perc": 0.15, 
                      "maintenance_perc": 0.031,"insurance_perc": 0.01,"consumption": 485, "crew": 3, 
                      "unloader_type": 'Mobile crane',"lifting_capacity": 60, "hourly_cycles": 30, "eff_fact": 0.55}

In [75]:
# create crane class (will be moved to Port Model revamp notebook)
class cyclic_unloader(cyclic_properties_mixin):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def total_capacity_calc(self):
        self.total_peak_capacity = int(self.quantity * self.peak_capacity)
        self.total_eff_capacity  = int(self.total_peak_capacity * self.eff_fact)
    
    def original_value_calc(self):
        self.original_value = int(self.quantity * self.unit_rate) 
        
    def lease_calc(self):
        if self.unloader_type == 'Mobile crane':
            self.lease = int(self.original_value * 0.10)
        else:
            return
        
    def maintenance_calc(self):
        self.maintenance = int(self.original_value * self.maintenance_perc)
        
    def insurance_calc(self):
        self.insurance = int(self.original_value * self.insurance_perc)
        
    def consumption_calc(self):
        if self.unloader_type != 'Mobile crane':
            self.consumption = int(0.3 * self.total_peak_capacity * operational_hours)
        if self.unloader_type == 'Mobile crane':
            self.consumption = int(485 * operational_hours)
        
    def shifts_calc(self):
        self.shifts = int(np.ceil(operational_hours * self.crew / 6.5))
        
    def delta_calc(self):
        self.delta = 0
        
    def calcs(self):
        self.total_capacity_calc()
        self.original_value_calc()
        self.lease_calc()
        self.maintenance_calc()
        self.insurance_calc()
        self.consumption_calc()
        self.shifts_calc()
        self.delta_calc()

In [77]:
# create objects
gantry_cranes   = cyclic_unloader(**gantry_data)
harbour_cranes  = cyclic_unloader(**harbour_crane_data)  
mobile_cranes   = cyclic_unloader(**mobile_crane_data) 

## Cranes (continuous)

In [76]:
# create continuous loader class
class continuous_properties_mixin(object):
    def __init__(self, ownership, lifespan, unit_rate, mobilisation_perc, maintenance_perc, insurance_perc, crew, unloader_type, peak_capacity, eff_fact, *args, **kwargs):
        super().__init__(*args, **kwargs)
        "initialize"
        self.ownership            = ownership
        self.lifespan             = lifespan
        self.unit_rate            = unit_rate
        self.mobilisation_perc    = mobilisation_perc
        self.mobilisation         = int(mobilisation_perc * unit_rate)
        self.maintenance_perc     = maintenance_perc
        self.insurance_perc       = insurance_perc
        self.crew                 = crew
        self.unloader_type        = unloader_type
        self.peak_capacity        = peak_capacity
        self.eff_fact             = eff_fact 

# Initial data set, data from Excel_input.xlsx
continuous_screw_data = {"ownership": 'Terminal operator', "lifespan": 30, "unit_rate": 6900000, "mobilisation_perc": 0.15, 
                         "maintenance_perc": 0.02, "insurance_perc": 0.01, "crew": 2,"unloader_type": 'Screw unloader', 
                         "peak_capacity": 700, "eff_fact": 0.55}

In [ ]:
# Create continuous crane class
class continuous_unloader(continuous_properties_mixin):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def total_capacity_calc(self):
        self.total_peak_capacity = int(self.quantity * self.peak_capacity)
        self.total_eff_capacity  = int(self.total_peak_capacity * self.eff_fact)
    
    def original_value_calc(self):
        self.original_value = int(self.quantity * self.unit_rate) 
        
    def maintenance_calc(self):
        self.maintenance = int(self.original_value * self.maintenance_perc)
        
    def insurance_calc(self):
        self.insurance = int(self.original_value * self.insurance_perc)
        
    def consumption_calc(self):
        self.consumption = (0.52 * self.total_peak_capacity * operational_hours)
        
    def shifts_calc(self):
        self.shifts = int(np.ceil(operational_hours * self.crew / 6.5))
        
    def delta_calc(self):
        self.delta = 0
        
    def calcs(self):
        self.total_capacity_calc()
        self.original_value_calc()
        self.maintenance_calc()
        self.insurance_calc()
        self.consumption_calc()
        self.shifts_calc()
        #self.delta_calc()

In [72]:
# Create objects
screw_unloaders = continuous_unloader(**continuous_screw_data)

## Conveyors

In [117]:
# Conveyor properties
class conveyor_properties_mixin(object):
    def __init__(self, ownership, lifespan, mobilisation, maintenance_perc, insurance_perc, crew, *args, **kwargs):
        super().__init__(*args, **kwargs)
        "initialize"
        self.ownership            = ownership
        self.lifespan             = lifespan
        self.mobilisation         = mobilisation
        self.maintenance_perc     = maintenance_perc
        self.insurance_perc       = insurance_perc
        self.crew                 = crew

# Initial data set, data from Excel_input.xlsx
conveyor_data = {"ownership": 'Terminal operator', "lifespan": 10, "mobilisation": 30000, 
                 "maintenance_perc": 0.10, "insurance_perc": 0.01, "crew": 0.5,}

In [120]:
# create conveyor class (will be moved to Port Model revamp notebook)
class conveyor(conveyor_properties_mixin):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def capacity_calc(self):
        self.capacity = int(np.ceil((gantry_cranes.peak_capacity   * gantry_cranes.quantity +\
                                     harbour_cranes.peak_capacity  * harbour_cranes.quantity +\
                                     mobile_cranes.peak_capacity   * mobile_cranes.quantity +\
                                     screw_unloaders.peak_capacity * screw_unloaders.quantity)/400)*400)
    
    def unit_rate_calc(self):
        self.unit_rate = int(6.0 * self.length)
    
    def original_value_calc(self):
        self.original_value = int(self.capacity * self.unit_rate) 
        
    def maintenance_calc(self):
        self.maintenance = int(self.original_value * self.maintenance_perc)
        
    def insurance_calc(self):
        self.insurance = int(self.original_value * self.insurance_perc)
        
    def consumption_calc(self):
        self.consumption = int(0.08 * self.capacity * operational_hours)
        
    def shifts_calc(self):
        self.shifts = int(np.ceil(operational_hours * self.crew / 6.5))
        
    def delta_calc(self):
        self.delta = int(np.ceil((gantry_cranes.peak_capacity   * gantry_cranes.delta  +\
                                  harbour_cranes.peak_capacity  * harbour_cranes.delta +\
                                  mobile_cranes.peak_capacity   * mobile_cranes.delta  +\
                                  screw_unloaders.peak_capacity * screw_unloaders.delta)/400)*400)
        
    def calcs(self):
        #self.capacity_calc()
        self.unit_rate_calc()
        self.original_value_calc()
        self.maintenance_calc()
        self.insurance_calc()
        self.consumption_calc()
        self.shifts_calc()
        #self.delta_calc()

In [122]:
# create objects   
conveyor_quay       = conveyor(**conveyor_data)
conveyor_hinterland = conveyor(**conveyor_data)

conveyor_quay.length       = 500
conveyor_hinterland.length = 500

conveyor_quay.calcs()
conveyor_hinterland.calcs()

## Storage

In [86]:
# create storage class
class storage_properties_mixin(object):
    def __init__(self, ownership, lifespan, unit_rate, mobilisation_min, mobilisation_perc, maintenance_perc, insurance_perc, storage_type, *args, **kwargs):
        super().__init__(*args, **kwargs)
        "initialize"
        self.ownership            = ownership
        self.lifespan             = lifespan
        self.unit_rate            = unit_rate
        self.mobilisation_min     = mobilisation_min
        self.mobilisation_perc    = mobilisation_perc
        self.maintenance_perc     = maintenance_perc
        self.insurance_perc       = insurance_perc
        self.storage_type         = storage_type
        
# Initial data set, data from Excel_input.xlsx
silo_data      = {"ownership": 'Terminal operator', "lifespan": 30, "unit_rate": 60, 
                  "mobilisation_min": 200000, "mobilisation_perc": 0.02, "maintenance_perc": 0.02, 
                  "insurance_perc": 0.01, "storage_type": 'Silos'}
warehouse_data = {"ownership": 'Terminal operator', "lifespan": 30, "unit_rate": 140,
                  "mobilisation_min": 200000, "mobilisation_perc": 0.02, "maintenance_perc": 0.01, 
                  "insurance_perc": 0.01, "storage_type": 'Warehouse'}

In [125]:
# create storage class (will be moved to Port Model revamp notebook)
class storage(storage_properties_mixin):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    
    def capacity_calc(self):
        self.capacity = 0    
    
    def original_value_calc(self):
        self.original_value = int(self.capacity * self.unit_rate) 
    
    def mobilisation_calc(self):
        self.mobilisation = int(max((self.delta * self.unit_rate * self.mobilisation_perc), self.mobilisation_min))
        
    def maintenance_calc(self):
        self.maintenance = int(self.original_value * self.maintenance_perc)
        
    def insurance_calc(self):
        self.insurance = int(self.original_value * self.insurance_perc)
        
    def consumption_calc(self):
        if self.storage_type == 'Silos':
            self.consumption = int(0.003 * self.capacity * operational_hours)
        if self.storage_type == 'Warehouse':
            self.consumption = int(0.001 * self.capacity * operational_hours)
        
    def shifts_calc(self):
        if self.storage_type == 'Silos':
            self.crew   = 0.00002 * self.capacity
        if self.storage_type == 'Warehouse':
            self.crew   = 0.00004 * self.capacity
        self.shifts = int(np.ceil(operational_hours * self.crew / 6.5))
        
    def calcs(self):
        self.capacity_calc()
        self.original_value_calc()
        self.mobilisation_calc()
        self.maintenance_calc()
        self.insurance_calc()
        self.consumption_calc()
        self.shifts_calc()

In [112]:
# create objects       
silos     = storage(**silo_data)
warehouse = storage(**warehouse_data)

## Loading Station

In [18]:
# create loading station class
class hinterland_station_properties_mixin(object):
    def __init__(self, ownership, lifespan, unit_rate, mobilisation, maintenance_perc, insurance_perc, crew, *args, **kwargs):
        super().__init__(*args, **kwargs)
        "initialize"
        self.ownership            = ownership
        self.lifespan             = lifespan
        self.unit_rate            = unit_rate
        self.mobilisation         = mobilisation
        self.maintenance_perc     = maintenance_perc
        self.insurance_perc       = insurance_perc
        self.crew                 = crew
        
# Initial data set, data from Excel_input.xlsx
hinterland_station_data = {"ownership": 'Terminal operator', "lifespan": 15, "unit_rate": 4000, "mobilisation": 100000, 
                           "maintenance_perc": 0.02, "insurance_perc": 0.01, "crew": 2}

In [126]:
# create storage class (will be moved to Port Model revamp notebook)
class hinterland_station(hinterland_station_properties_mixin):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def capacity_calc(self):
        self.capacity = 0    
    
    def original_value_calc(self):
        self.original_value = int(self.capacity * self.unit_rate) 
        
    def maintenance_calc(self):
        self.maintenance = int(self.original_value * self.maintenance_perc)
        
    def insurance_calc(self):
        self.insurance = int(self.original_value * self.insurance_perc)
        
    def consumption_calc(self):
        self.consumption = int(0.75 * self.capacity * operational_hours)
        
    def shifts_calc(self):
        self.shifts = int(np.ceil(operational_hours * self.crew / 6.5))
        
    def calcs(self):
        self.capacity_calc()
        self.original_value_calc()
        self.maintenance_calc()
        self.insurance_calc()
        self.consumption_calc()
        self.shifts_calc()

In [130]:
# create objects
loading_station = hinterland_station(**hinterland_station_data)

# Business Logic